In [1]:
from sklearn.ensemble import RandomForestClassifier as RFC_
from sklearn.ensemble import RandomForestRegressor as RFR_

In [2]:
import time
ini_time=time.time()

import datetime

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np




#%run 'prev_comex_ncm.ipynb'

# Define parametros
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
l=20

comex_ncm=pd.read_csv('~/git/BD/temp/imp_fert_quanti.csv').sort_values(by=['CO_ANO','CO_MES'])

# Filtra top
top_nick=comex_ncm.groupby('nick').KG_LIQUIDO.sum().sort_values().index[-2:]
top_uf=comex_ncm.groupby('SG_UF_NCM').KG_LIQUIDO.sum().sort_values().index[-2:]
top_pais=comex_ncm.groupby('CO_PAIS').KG_LIQUIDO.sum().sort_values().index[-2:]
top_urf=comex_ncm.groupby('CO_URF').KG_LIQUIDO.sum().sort_values().index[-2:]
comex_ncm=comex_ncm[(comex_ncm.nick.isin(top_nick))&(comex_ncm.SG_UF_NCM.isin(top_uf))&(comex_ncm.CO_PAIS.isin(top_pais))&(comex_ncm.CO_URF.isin(top_urf))]

# Salva datas de início e fim
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
date_ini=comex_ncm.date.min()
date_fim=comex_ncm.date.max()

In [3]:
# Sparse Matrix 
# Matriz 1
anos=pd.DataFrame({'CO_ANO':np.arange(comex_ncm.CO_ANO.max(),comex_ncm.CO_ANO.max()+1),'key':0})
# Matriz 2
cat=comex_ncm[cat_cols].drop_duplicates().assign(key=0)
# Multiplica
c_c=anos.merge(cat,how='outer').drop(columns='key')        
comex_ncm=c_c.merge(comex_ncm,how='outer').fillna(0)
# Refaz data
comex_ncm['date']=pd.to_datetime(comex_ncm.CO_ANO.astype(str)+'/'+comex_ncm.CO_MES.astype(str)+'/1')
# Apara as bordas
comex_ncm=comex_ncm[(comex_ncm.date<=date_fim)&(comex_ncm.date>=date_ini)]

In [4]:
comex_ncm

,CO_ANO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,KG_LIQUIDO,date
0,1997,8,MOP_k,PR,999,817800,3045000.0,1997-08-01
1,1998,7,MOP_k,PR,676,817800,2346999.0,1998-07-01
2,2004,1,UREIA_k,PR,999,817800,2000000.0,2004-01-01
3,2004,12,UREIA_k,PR,999,817800,2008950.0,2004-12-01
4,2007,5,MOP_k,PR,676,817800,2459150.0,2007-05-01
...,...,...,...,...,...,...,...,...
575,2024,4,UREIA_k,PR,676,817800,0.0,2024-04-01
576,2024,4,UREIA_k,PR,676,917800,6524000.0,2024-04-01
577,2024,4,UREIA_k,PR,999,917800,22839957.0,2024-04-01
578,2024,4,UREIA_k,RS,676,917800,0.0,2024-04-01


In [5]:
comex_ncm.date.min

<bound method Series.min of 0     1997-08-01
1     1998-07-01
2     2004-01-01
3     2004-12-01
4     2007-05-01
         ...    
575   2024-04-01
576   2024-04-01
577   2024-04-01
578   2024-04-01
579   2024-04-01
Name: date, Length: 580, dtype: datetime64[ns]>

In [6]:
target_cols=['oc','KG_LIQUIDO']
cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF']
md_c=15
md_r=1

# Identifica as datas de previsao
pred_max=comex_ncm.date.max()
pred_min=comex_ncm.date.min()
# Classificação ##########################################################################################

# Marca ocorrências
comex_ncm=comex_ncm.assign(oc=0)
comex_ncm.loc[comex_ncm[target_cols[1]]>0,'oc']=1
comex_ncm

,CO_ANO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,KG_LIQUIDO,date,oc
0,1997,8,MOP_k,PR,999,817800,3045000.0,1997-08-01,1
1,1998,7,MOP_k,PR,676,817800,2346999.0,1998-07-01,1
2,2004,1,UREIA_k,PR,999,817800,2000000.0,2004-01-01,1
3,2004,12,UREIA_k,PR,999,817800,2008950.0,2004-12-01,1
4,2007,5,MOP_k,PR,676,817800,2459150.0,2007-05-01,1
...,...,...,...,...,...,...,...,...,...
575,2024,4,UREIA_k,PR,676,817800,0.0,2024-04-01,0
576,2024,4,UREIA_k,PR,676,917800,6524000.0,2024-04-01,1
577,2024,4,UREIA_k,PR,999,917800,22839957.0,2024-04-01,1
578,2024,4,UREIA_k,RS,676,917800,0.0,2024-04-01,0


In [7]:
# Codifica categorias
comex_ncm[cat_cols]=comex_ncm[cat_cols].astype('category')    
comex_ncm_dumm=pd.get_dummies(comex_ncm, prefix_sep='~')

# Data em formato para processamento
comex_ncm_dumm.date=comex_ncm_dumm.date.astype(int)
# Ano substituido por date, que da a sequencia dos meses
comex_ncm_dumm.drop(columns='CO_ANO',inplace=True)

comex_ncm_dumm

,KG_LIQUIDO,date,oc,CO_MES~1,CO_MES~2,CO_MES~3,CO_MES~4,CO_MES~5,CO_MES~6,CO_MES~7,...,CO_MES~11,CO_MES~12,nick~MOP_k,nick~UREIA_k,SG_UF_NCM~PR,SG_UF_NCM~RS,CO_PAIS~676,CO_PAIS~999,CO_URF~817800,CO_URF~917800
0,3045000.0,870393600000000000,1,False,False,False,False,False,False,False,...,False,False,True,False,True,False,False,True,True,False
1,2346999.0,899251200000000000,1,False,False,False,False,False,False,True,...,False,False,True,False,True,False,True,False,True,False
2,2000000.0,1072915200000000000,1,True,False,False,False,False,False,False,...,False,False,False,True,True,False,False,True,True,False
3,2008950.0,1101859200000000000,1,False,False,False,False,False,False,False,...,False,True,False,True,True,False,False,True,True,False
4,2459150.0,1177977600000000000,1,False,False,False,False,True,False,False,...,False,False,True,False,True,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575,0.0,1711929600000000000,0,False,False,False,True,False,False,False,...,False,False,False,True,True,False,True,False,True,False
576,6524000.0,1711929600000000000,1,False,False,False,True,False,False,False,...,False,False,False,True,True,False,True,False,False,True
577,22839957.0,1711929600000000000,1,False,False,False,True,False,False,False,...,False,False,False,True,True,False,False,True,False,True
578,0.0,1711929600000000000,0,False,False,False,True,False,False,False,...,False,False,False,True,False,True,True,False,False,True


In [8]:
pred_min

Timestamp('1997-08-01 00:00:00')

In [9]:
pred_max

Timestamp('2024-04-01 00:00:00')

In [10]:
pred_min

Timestamp('1997-08-01 00:00:00')

In [11]:
# Separa train, test_max e test_max
def train_test_max_test_min(df, pred_max, pred_min):
    '''Identifica train, test_max, test_min'''
    
    train=df[(df.date.astype('datetime64[ns]')>pred_min)&(df.date.astype('datetime64[ns]')<pred_max)]
    # Identifica test na data mínima e máxima
    test_max=df[df.date.astype('datetime64[ns]')==pred_max]
    test_min=df[df.date.astype('datetime64[ns]')==pred_min]    
    return [train,test_max,test_min]
[train_c,test_max_c,test_min_c]=train_test_max_test_min(comex_ncm_dumm,pred_max, pred_min)

In [12]:
# Separa em X e y
def Xy(train,test_max, test_min, col_target,col_del):
    X_train=train.drop(columns=[col_target]+[col_del])
    y_train=train[col_target]
    X_test_max=test_max.drop(columns=[col_target]+[col_del])
    y_test_max=test_max[col_target]
    X_test_min=test_min.drop(columns=[col_target]+[col_del])
    y_test_min=test_min[col_target]
    return [X_train,y_train,X_test_max,y_test_max,X_test_min,y_test_min]

[X_train_c,y_train_c,X_test_max_c,y_test_max_c,X_test_min_c,y_test_min_c]=Xy(train_c,test_max_c,test_min_c,target_cols[0],target_cols[1])
# Prediz

In [13]:
def fit_pred(RF,md,X_train,y_train,X_test_max,y_test_max,X_test_min,y_test_min, col_target):
    # Fit
    RF=RF(max_depth=md)
    RF.fit(X_train,y_train)

    # Prediz Max
    test_pred_max=X_test_max.assign(col1=RF.predict(X_test_max)).rename(columns={'col1':str(col_target)+'_pred'})
    test_pred_max=test_pred_max.assign(col2=y_test_max).rename(columns={'col2':(str(col_target))})

    # Prediz min
    test_pred_min=X_test_min.assign(col1=RF.predict(X_test_min)).rename(columns={'col1':str(col_target)+'_pred'})
    test_pred_min=test_pred_min.assign(col2=y_test_min).rename(columns={'col2':(str(col_target))})

    return [test_pred_max,test_pred_max]

In [14]:
list_test_pred_c=fit_pred(RFC_,md_c,X_train_c,y_train_c,X_test_max_c,y_test_max_c,X_test_min_c,y_test_min_c,target_cols[0])

test_pred_max_c,test_pred_min_c=list_test_pred_c

In [15]:
# Regressão ##########################################################################################

# Filtra deixando passar somente ocorridos e previstos de ocorrer
train_r=train_c[train_c.oc==1]
test_max_r=test_pred_max_c[(test_pred_max_c.oc==1)&(test_pred_max_c.oc_pred==1)]
test_max_r=test_pred_min_c[(test_pred_min_c.oc==1)&(test_pred_min_c.oc_pred==1)]

In [16]:
#if (len(train_r)>0) & ((len(test_max_r)>0) | (len(test_max_r)>0)):

# Filtra deixando passar para o treino somente combinacoes categoricas presentes na predicao
def decode(df):
    df=pd.concat([
    df.select_dtypes(exclude='bool'),
    pd.from_dummies(df.select_dtypes(include='bool'),sep='~')],axis=1)
    return df
train_r=decode(train_r)
test_max_r=decode(test_max_r)
test_max_r=decode(test_max_r)

In [17]:
filtro=pd.concat([test_max_r,test_max_r])[cat_cols].drop_duplicates()
train_r=train_r.merge(filtro)
train_r.head(1)

,KG_LIQUIDO,date,oc,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,2500000.0,1238544000000000000,1,4,UREIA_k,PR,676,917800


In [18]:
comex_ncm.date=comex_ncm.date.astype('int')
test_max_r=test_max_r.merge(comex_ncm)
test_max_r=test_max_r.merge(comex_ncm)

In [19]:
test_max_r

,date,oc_pred,oc,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF,CO_ANO,KG_LIQUIDO


In [ ]:
comex_ncm_r=pd.concat([train_r,test_max_r,test_max_r])

In [21]:
# Codifica categorias
comex_ncm[cat_cols]=comex_ncm[cat_cols].astype('category')    
comex_ncm_dumm=pd.get_dummies(comex_ncm, prefix_sep='~')

comex_ncm_dumm.date=comex_ncm_dumm.date.astype(int)

In [22]:
# Separa train and tests
[train_r,test_max_r,test_min_r]=train_test_max_test_min(comex_ncm_dumm,pred_max, pred_min)

In [23]:
decode(test_min_r)

,CO_ANO,KG_LIQUIDO,date,oc,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,1997,3045000.0,870393600000000000,1,8,MOP_k,PR,999,817800


In [24]:
# Separa X e y
[X_train_r,y_train_r,X_test_max_r,y_test_max_r,X_test_min_r,y_test_min_r]=Xy(train_r,test_max_r,test_min_r,target_cols[1],target_cols[0])

In [38]:
decode(X_test_min_r)

,CO_ANO,date,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,1997,870393600000000000,8,MOP_k,PR,999,817800


In [ ]:
# Prediz
list_test_pred_r=fit_pred(RFR_,md_r,X_train_r,y_train_r,X_test_max_r,y_test_max_r,X_test_min_r,y_test_min_r,target_cols[1])

In [35]:
list_test_pred_r

[     CO_ANO                 date  CO_MES~1  CO_MES~2  CO_MES~3  CO_MES~4  \
 573    2024  1711929600000000000     False     False     False      True   
 574    2024  1711929600000000000     False     False     False      True   
 575    2024  1711929600000000000     False     False     False      True   
 576    2024  1711929600000000000     False     False     False      True   
 577    2024  1711929600000000000     False     False     False      True   
 578    2024  1711929600000000000     False     False     False      True   
 579    2024  1711929600000000000     False     False     False      True   
 
      CO_MES~5  CO_MES~6  CO_MES~7  CO_MES~8  ...  nick~MOP_k  nick~UREIA_k  \
 573     False     False     False     False  ...        True         False   
 574     False     False     False     False  ...        True         False   
 575     False     False     False     False  ...       False          True   
 576     False     False     False     False  ...       False     

In [ ]:
test_pred_max_r,test_pred_min_r=list_test_pred_r

In [28]:
decode(test_pred_min_r)

,CO_ANO,date,KG_LIQUIDO_pred,KG_LIQUIDO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
573,2024,1711929600000000000,2.455242e+07,52606851.0,4,MOP_k,PR,676,917800
574,2024,1711929600000000000,2.455242e+07,16800000.0,4,MOP_k,PR,999,917800
575,2024,1711929600000000000,1.763877e+07,0.0,4,UREIA_k,PR,676,817800
576,2024,1711929600000000000,1.874277e+07,6524000.0,4,UREIA_k,PR,676,917800
577,2024,1711929600000000000,1.874277e+07,22839957.0,4,UREIA_k,PR,999,917800
578,2024,1711929600000000000,1.227664e+07,0.0,4,UREIA_k,RS,676,917800
579,2024,1711929600000000000,1.227664e+07,800000.0,4,UREIA_k,RS,999,917800


In [29]:
#else:
#    test_pred_max_r,test_pred_max_r=pd.DataFrame(), pd.DataFrame()

# Junta classi e regress resultados
test_pred_c=pd.concat([test_pred_max_c,test_pred_min_c])
test_pred_r=pd.concat([test_pred_max_r,test_pred_min_r])

In [30]:
decode(test_pred_r).reset_index(drop=True)

,CO_ANO,date,KG_LIQUIDO_pred,KG_LIQUIDO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,2024,1711929600000000000,2.455242e+07,52606851.0,4,MOP_k,PR,676,917800
1,2024,1711929600000000000,2.455242e+07,16800000.0,4,MOP_k,PR,999,917800
2,2024,1711929600000000000,1.763877e+07,0.0,4,UREIA_k,PR,676,817800
3,2024,1711929600000000000,1.874277e+07,6524000.0,4,UREIA_k,PR,676,917800
4,2024,1711929600000000000,1.874277e+07,22839957.0,4,UREIA_k,PR,999,917800
5,2024,1711929600000000000,1.227664e+07,0.0,4,UREIA_k,RS,676,917800
6,2024,1711929600000000000,1.227664e+07,800000.0,4,UREIA_k,RS,999,917800
7,2024,1711929600000000000,2.455242e+07,52606851.0,4,MOP_k,PR,676,917800
8,2024,1711929600000000000,2.455242e+07,16800000.0,4,MOP_k,PR,999,917800
9,2024,1711929600000000000,1.763877e+07,0.0,4,UREIA_k,PR,676,817800


In [31]:
test_pred=test_pred_c.merge(test_pred_r)

In [32]:
test_pred=decode(test_pred)
                     
test_pred

,date,oc_pred,oc,CO_ANO,KG_LIQUIDO_pred,KG_LIQUIDO,CO_MES,nick,SG_UF_NCM,CO_PAIS,CO_URF
0,1711929600000000000,1,1,2024,2.455242e+07,52606851.0,4,MOP_k,PR,676,917800
1,1711929600000000000,1,1,2024,2.455242e+07,52606851.0,4,MOP_k,PR,676,917800
2,1711929600000000000,1,1,2024,2.455242e+07,16800000.0,4,MOP_k,PR,999,917800
3,1711929600000000000,1,1,2024,2.455242e+07,16800000.0,4,MOP_k,PR,999,917800
4,1711929600000000000,0,0,2024,1.763877e+07,0.0,4,UREIA_k,PR,676,817800
5,1711929600000000000,0,0,2024,1.763877e+07,0.0,4,UREIA_k,PR,676,817800
6,1711929600000000000,1,1,2024,1.874277e+07,6524000.0,4,UREIA_k,PR,676,917800
7,1711929600000000000,1,1,2024,1.874277e+07,6524000.0,4,UREIA_k,PR,676,917800
8,1711929600000000000,1,1,2024,1.874277e+07,22839957.0,4,UREIA_k,PR,999,917800
9,1711929600000000000,1,1,2024,1.874277e+07,22839957.0,4,UREIA_k,PR,999,917800


In [33]:
print("preve_comex_ncm(comex_ncm,target_cols=['oc','KG_LIQUIDO'],cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF'],md_c=15,md_r=1):")

preve_comex_ncm(comex_ncm,target_cols=['oc','KG_LIQUIDO'],cat_cols=['CO_MES','nick','SG_UF_NCM','CO_PAIS','CO_URF'],md_c=15,md_r=1):


In [34]:
print("RandomForestClassifier as RFC_ & RandomForestRegressor as RFR_")

RandomForestClassifier as RFC_ & RandomForestRegressor as RFR_
